In [ ]:
%load_ext autoreload
%autoreload 2
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import argparse
import repackage

import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
from experiment_utils import set_env, seed_everything
set_env()
from cgn_extensions.mnists.dataloaders import get_dataloaders

# Train concept classifier for double-colored-MNIST
We need to train:
1. A CF for classifying the digit's color
2. A CF for classifying the digit's shape
3. A CF for background color

## Load dataset

In [ ]:
dl_train, dl_test = get_dataloaders('double_colored_MNIST', batch_size=1000, workers=4)

### Dataset size

In [ ]:
print(f"Train: {dl_train.dataset.labels.shape[0]}")
print(f"Test: {dl_test.dataset.labels.shape[0]}")

### Get a sample image

In [ ]:
random_idx = np.random.randint(0, dl_train.dataset.labels.shape[0])

In [ ]:
plt.imshow(dl_train.dataset.__getitem__(random_idx)['ims'].transpose(0,1).transpose(1,2))
label = dl_train.dataset.__getitem__(random_idx)['labels']
plt.title(f"Digit: {label}")
plt.show()
print(f"BG label: {dl_train.dataset.__getitem__(random_idx)['bg_labels']}")
print(f"Texture label: {dl_train.dataset.__getitem__(random_idx)['texture_labels']}")

## Train color CF

In [ ]:
from cgn_extensions.mnists.models.mnist_color_cf import train, test, MNIST_COLOR_CNN

# Args for training
args = argparse.Namespace()
args.batch_size = 512
args.gamma = 0.7
args.epochs = 10
args.lr = 0.01
args.log_interval = 10
args.dataset = 'double_colored_MNIST'

# Data
dl_train, dl_test = get_dataloaders(args.dataset, batch_size=args.batch_size, workers=4)

# Model
model = MNIST_COLOR_CNN(num_classes=10)

# Optimizer
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

# push to device and train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
device

In [ ]:
train_accs = dict()
test_accs = dict()
for epoch in range(1, args.epochs + 1):
    train_acc = train(args, model, device, dl_train, optimizer, epoch, 'texture', max_batches=200)
    test_acc = test(model, device, dl_test, 'texture')
    train_accs[epoch] = train_acc
    test_accs[epoch] = test_acc
    scheduler.step()

In [ ]:
# Plot losses
plt.plot(train_accs.keys(), train_accs.values())
plt.plot(test_accs.keys(), test_accs.values())
plt.legend(['train', 'test'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

### GradCam

In [ ]:
# Target layer for Grad-CAM
target_layers = [model.model[3]]

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available())

# Get a random batch
dl_train, dl_test = get_dataloaders('double_colored_MNIST', batch_size=10, workers=4)
gradcam_data = next(iter(dl_train))
input_tensor = gradcam_data['ims'][0].unsqueeze(0).clip(min=0, max=1).to(device)
grayscale_cam = cam(input_tensor=input_tensor, targets=None)
grayscale_cam = grayscale_cam[0, :]

rgb_img = input_tensor.squeeze(0).transpose(0,1).transpose(1,2).to('cpu').detach().numpy()
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

In [ ]:
plt.imshow(rgb_img)

In [ ]:
plt.imshow(visualization)

### Save the model

In [ ]:
# # save the model
# dataset_suffix = (args.dataset) if not args.combined else (args.dataset + "_combined")
# dataset_suffix += "_seed_" + str(args.seed) if args.seed is not None else ""
# save_path = f'mnists/experiments/classifier_{dataset_suffix}/weights/ckp_epoch_{args.epochs}.pth'
# os.makedirs(os.path.dirname(save_path), exist_ok=True)
# print('Saving model to {}'.format(save_path))
# torch.save(model.state_dict(), save_path)

### Make some predictions

## Train shape CF

## Train background CF